<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/ml_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [218]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [219]:
from IPython.display import SVG, display, HTML

In [220]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [244]:
import spacy

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [222]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [223]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content		 new_anns60k.csv.1  new_coref_df_anns60k.csv  sample_data
new_anns60k.csv  new_anns60k.csv.2  new_df_anns60k.csv
MyDrive
.shortcut-targets-by-id
.file-revisions-by-id
.Trash


In [224]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [225]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-23 14:20:03--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt’

neuroleptic_antidep 100%[===================>]  11.12M  16.7MB/s    in 0.7s    

2021-05-23 14:20:05 (16.7 MB/s) - ‘neuroleptic_antidepr2.txt’ saved [11655409/11655409]

--2021-05-23 14:20:05--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [226]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-23 14:20:10--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-23 14:20:11 ERROR 404: Not Found.

--2021-05-23 14:20:11--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2045566 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv’

annotation_table.cs 100%[===================>]   1.95M  8.61MB/s    in 0.2s

In [227]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [236]:
posts = my_txt.split('----')

## **Обучение векторизатора**

In [245]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]

vectorized_txt = tfidf.fit_transform(sentences_tokenized)
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [260]:
vvv = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      vvv +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 18:
        new_row = new_row.split('\t')[:18]
      writer.writerow(new_row)
vvv

27019

In [261]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [262]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
# !ls

In [263]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,7.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0
27014,таможенные,0,22,68.0,0.0,8.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0
27015,пошлины,0,22,68.0,0.0,9.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0
27016,как,0,22,68.0,0.0,10.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0


In [265]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [ ]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [299]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [300]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [301]:
anns_df['SENT_VECT '] = sent_vect_list

In [302]:
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,"[0.19743444, 0.21119584, 0.09573975, -0.425207...","[0.19743444, 0.21119584, 0.09573975, -0.425207...","[0.18557024, 0.35647067, 0.20470199, -0.290656..."
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,"[0.22798513, 0.5543274, -0.20923378, -0.080853...","[0.19743444, 0.21119584, 0.09573975, -0.425207...","[0.18557024, 0.35647067, 0.20470199, -0.290656..."
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,"[0.43248653, 0.5937064, 0.34339568, -0.0548174...","[0.22798513, 0.5543274, -0.20923378, -0.080853...","[0.18557024, 0.35647067, 0.20470199, -0.290656..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,"[0.46160558, 0.52094615, 0.6171367, -0.1088908...","[0.43248653, 0.5937064, 0.34339568, -0.0548174...","[0.18557024, 0.35647067, 0.20470199, -0.290656..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,"[0.35837942, 0.58045554, 0.68698806, -0.198242...","[0.46160558, 0.52094615, 0.6171367, -0.1088908...","[0.18557024, 0.35647067, 0.20470199, -0.290656..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,0,22,68.0,0.0,7.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,"[0.4180836, -0.024044996, 0.039385114, -0.3076...","[-0.34927273, -0.6403187, -0.4151997, -0.01996...","[0.10717429, 0.3408835, 0.28401107, -0.3589822..."
26493,таможенные,0,22,68.0,0.0,8.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,"[-0.07314223, -1.1396724, -0.84783167, -0.9857...","[-0.34927273, -0.6403187, -0.4151997, -0.01996...","[0.10717429, 0.3408835, 0.28401107, -0.3589822..."
26494,пошлины,0,22,68.0,0.0,9.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,"[-0.34927273, -0.6403187, -0.4151997, -0.01996...","[-0.25378042, 0.5518586, 0.41950038, -0.919631...","[0.10717429, 0.3408835, 0.28401107, -0.3589822..."
26495,как,0,22,68.0,0.0,10.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,"[0.5683889, 0.81005967, 0.5475652, -1.6073513,...","[-0.25378042, 0.5518586, 0.41950038, -0.919631...","[0.10717429, 0.3408835, 0.28401107, -0.3589822..."


In [ ]:
topics = list(set(df.TOPIC_NUM.values)) 
for topic_num in topics:
  # if topic_num == 2:
  #   break
  # извлекаем предложения топика
  sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
  for sent_num in sents_in_topic:
    sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

Проверим данные

In [ ]:
anns_df

In [183]:
import numpy as np

In [185]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [216]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [217]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [188]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [213]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [ ]:
cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [214]:
df = cor_df.iloc[:15000,:]

In [215]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER,ANN_TOKEN
0,обсуждаем,"[0.05871739, 0.34252143, -0.08884978, -0.74504...",0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,"[0.05871739, 0.34252143, -0.08884978, -0.74504...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[-0.7033859, 0.7749181, 0.17049398, 0.19153355...",0,0.0,обсуждаем
1,препарат,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...",0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[0.05871739, 0.34252143, -0.08884978, -0.74504...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[-0.7033859, 0.7749181, 0.17049398, 0.19153355...",0,0.0,препарат
2,рисперидон,"[0.61300194, 0.6183907, 0.25589323, -0.4421036...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.61300194, 0.6183907, 0.25589323, -0.4421036...",1,2.0,рисперидон
3,рисполепт,"[0.4014503, 0.6873261, 0.49922436, -0.5146114,...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.61300194, 0.6183907, 0.25589323, -0.4421036...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.4014503, 0.6873261, 0.49922436, -0.5146114,...",1,3.0,рисполепт
4,сперидан,"[0.0651561394, 0.543457031, 0.580361426, -0.40...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.4014503, 0.6873261, 0.49922436, -0.5146114,...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.0217187125, 0.181152344, 0.193453804, -0.13...",1,4.0,сперидан
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,"[0.51364344, 0.02838964, -0.06961589, -0.70076...",0,16,3,0,61,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,"[0.202230185, -0.0267577451, 0.0462362356, -0....","[0.02458156, 0.01716435, 0.00043265, 0.0057551...","[0.51364344, 0.02838964, -0.06961589, -0.70076...",1,2215.0,абраксан
14996,вакцина,"[0.202230185, -0.0267577451, 0.0462362356, -0....",0,16,3,0,62,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,"[-0.08942804, -0.19534485, 0.34863153, -0.2930...","[0.02458156, 0.01716435, 0.00043265, 0.0057551...","[-0.05610157, 0.6278774, 0.33735275, -0.344426...",0,0.0,с
14997,аукнется,"[-0.08942804, -0.19534485, 0.34863153, -0.2930...",0,16,3,0,63,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,"[0.3954894, 0.05844666, 0.47036946, -0.2928081...","[0.02458156, 0.01716435, 0.00043265, 0.0057551...","[0.0390081294, -0.053258121, 0.118922442, -0.4...",1,2217.0,гемцитабином
14998,на,"[0.421984941, 0.303224176, 0.836256504, -0.617...",0,16,3,0,64,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,"[0.33309454, 0.70234656, 0.59898806, 0.5915255...","[0.02458156, 0.01716435, 0.00043265, 0.0057551...","[0.421984941, 0.303224176, 0.836256504, -0.617...",0,0.0,в


Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [192]:
len(df.loc[df['NER'] == 1])

579

Из них местоимений (прономинальные анафоры).

In [193]:
len(df.loc[df['NER'] == 3])

112

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [194]:
len(df.loc[df['NER'] == 2])

215

In [195]:
len(df.loc[df['NER'].isin([1,2,3])])

906

In [196]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [197]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.61300194, 0.6183907, 0.25589323, -0.4421036...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.61300194, 0.6183907, 0.25589323, -0.4421036...",1,2.0
3,рисполепт,"[0.4014503, 0.6873261, 0.49922436, -0.5146114,...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.61300194, 0.6183907, 0.25589323, -0.4421036...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.4014503, 0.6873261, 0.49922436, -0.5146114,...",1,3.0
4,сперидан,"[0.0651561394, 0.543457031, 0.580361426, -0.40...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.4014503, 0.6873261, 0.49922436, -0.5146114,...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.0217187125, 0.181152344, 0.193453804, -0.13...",1,4.0
21,лекарством,"[0.18107422, 0.10991222, 0.40389267, 0.1257452...",0,1,1,1,21,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,"[1.1023946, -0.21851613, 0.27732524, -0.410499...","[0.0144689651, 0.0110738566, 0.0359230712, 0.0...","[0.15912572, 0.25575718, 0.47969347, -0.225838...",2,0.0
142,рисполепта,"[0.41737685, 0.53507644, 0.6493534, -0.4645206...",0,1,1,13,142,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[0.21620364, 0.55475205, 0.30858138, 0.0875703...","[0.05283712, 0.04810272, 0.0337651, -0.0647213...","[0.4274913, 0.616717, 0.4384798, -0.31317738, ...",1,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14921,случае,"[-0.8581553, -0.08993252, 0.46717307, -0.30277...",1,15,19,5,2329,inan,loct,masc,NaN,NOUN,obl,было,NaN,NaN,neut,VERB,"[0.2452363, 1.0681885, 0.560035, -0.47977963, ...","[0.05884434, 0.17941733, 0.09611521, -0.085921...","[0.01843335, 0.06042727, 0.14710088, -0.175997...",3,2138.0
14925,но,"[0.02900389, 0.62579334, 0.1714195, -0.1735671...",1,15,19,6,2333,NaN,NaN,NaN,NaN,CONJ,cc,рпж,NaN,NaN,NaN,NaN,"[0.33568636, 0.14108977, 0.18520364, -0.061363...","[0.08642551, 0.02916948, 0.0604148, -0.0244103...","[0.35438251, 0.29434806, 0.2716353, -0.2049615...",1,2145.0
14947,мочевого,"[0.30769342, 0.36641997, 0.6807388, -0.2809179...",0,16,1,0,13,NaN,gent,masc,NaN,ADJF,amod,пузыря,inan,gent,masc,NOUN,"[-0.07557568, 0.13845481, 0.5604058, 0.0803413...","[-0.00333615, 0.02435627, 0.08215391, 0.005502...","[0.30769342, 0.36641997, 0.6807388, -0.2809179...",1,2167.0
14995,эта,"[0.51364344, 0.02838964, -0.06961589, -0.70076...",0,16,3,0,61,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,"[0.202230185, -0.0267577451, 0.0462362356, -0....","[0.02458156, 0.01716435, 0.00043265, 0.0057551...","[0.51364344, 0.02838964, -0.06961589, -0.70076...",1,2215.0


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [198]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.61300194, 0.6183907, 0.25589323, -0.4421036...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.0328866281, 0.142481059, 0.138332754, -0.10...","[0.61300194, 0.6183907, 0.25589323, -0.4421036...",1,2.0
576,зелдокс,"[0.506304026, 0.684155226, -0.141768411, -0.25...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.09821402, 0.14675905, 0.10349324, -0.055811...","[0.506304026, 0.684155226, -0.141768411, -0.25...",1,2.0
1870,оланзапин,"[0.69098365, 0.84822035, 0.2730716, -0.3114577...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.04047806, 0.05509917, 0.08760619, -0.031909...","[0.69098365, 0.84822035, 0.2730716, -0.3114577...",0,0.0
5555,галоперидол,"[0.56483144, 0.9214677, -0.03640855, -0.353968...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[0.05871739, 0.34252143, -0.08884978, -0.74504...","[0.11229787, 0.4059439, 0.00351686, -0.1243023...","[-0.06927723, 0.84819293, 0.06704271, -0.08121...",0,0.0
6200,тиаприд,"[0.5601152, 0.43141252, 0.26385978, -0.3580163...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.16456556, 0.18610603, 0.06807116, -0.096986...","[0.5601152, 0.43141252, 0.26385978, -0.3580163...",0,0.0
8450,сероквель,"[0.41217506, 0.9760998, 0.24672835, -0.1187952...",0,6,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.7033859, 0.7749181, 0.17049398, 0.19153355...","[0.1290171, 0.32225966, 0.07695462, 0.01664167...","[0.41217506, 0.9760998, 0.24672835, -0.1187952...",0,0.0
8541,день,"[0.0313554741, 1.13139975, -0.552433074, -0.44...",0,7,0,0,2,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,"[0.0313554741, 1.13139975, -0.552433074, -0.44...","[0.105745733, 0.553744495, -0.282059699, -0.16...","[0.4499723, 0.7738633, -0.30182838, -0.0148975...",0,0.0
11117,500,"[0.3767316, -0.05859598, 0.92690504, -0.672322...",0,8,0,0,2,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,"[0.17310344, 0.6202443, 0.8790039, -0.30582935...","[-0.0276589, -0.02779311, 0.02321921, -0.00313...",[0.0],0,0.0
11787,капецитабин,"[0.34493223, 1.3637387, 0.6850874, -0.37695974...",0,10,0,0,2,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,"[0.34493223, 1.3637387, 0.6850874, -0.37695974...","[-0.0151779177, -0.0666496158, 0.0428031757, -...","[0.23536643, 0.30812463, 0.33023566, -0.504914...",0,0.0
11806,иресса,"[0.34256947, 0.46005294, 0.75681233, -0.733530...",0,11,0,0,2,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,"[0.5564224, 0.51623684, 0.10405769, -0.6757818...","[0.09474397, 0.05582693, 0.09183251, 0.0153907...","[0.06600111, 0.39540923, 0.91943175, -0.463366...",0,0.0


In [199]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [200]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [201]:
df.TOKEN_VECT[710]

array([nan])

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [202]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [203]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [204]:
df.TOKEN[1]

'препарат'

In [205]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на данные.

In [206]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.462631,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.497135,0.831452,0.612037,0,0.0
1,препарат,0.537895,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.497135,0.831452,0.612037,0,0.0
2,рисперидон,0.798992,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.874818,0.831452,0.819550,1,2.0
3,рисполепт,0.827964,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.595599,0.831452,0.841689,1,3.0
4,сперидан,0.741854,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.686281,0.831452,0.790878,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,0.396652,0,16,3,0,61,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,0.468116,0.464917,0.399088,1,2215.0
14996,вакцина,0.506309,0,16,3,0,62,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,0.587453,0.464917,0.726396,0,0.0
14997,аукнется,0.519965,0,16,3,0,63,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,0.541723,0.464917,0.595201,1,2217.0
14998,на,0.444298,0,16,3,0,64,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,0.475042,0.464917,0.408353,0,0.0


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [207]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [208]:
df.iloc[[11696]]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
11696,спасибо,0.455624,1,9,2,2,527,NaN,NaN,NaN,NaN,INTJ,parataxis,ссылочка,inan,nomn,femn,NOUN,0.336119,0.52036,0.670402,0,0.0


In [209]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)].iloc[[2]]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
13,практически,0.551951,0,1,1,1,13,NaN,NaN,NaN,NaN,ADVB,advmod,нет,NaN,NaN,NaN,PRED,0.549618,0.489342,0.656121,0,0.0


In [210]:
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)].iloc[[2]].POST_NUM.values[0]

1

In [211]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
21,лекарством,0.427442,0,1,1,1,21,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,0.549618,0.489342,0.656121,2,0.0


In [212]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
for topic_num in topics:
  # if topic_num == 2:
  #   break
  # извлекаем предложения топика
  sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
  for sent_num in sents_in_topic:
    sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]
    # собираем анафоры
    current_anaphors = sent.loc[(sent['NER'] > 1)]
    print(current_anaphors)
    if sent_num == 3:
      break



        TOKEN  TOKEN_VECT  ...  NER  COREFERENCE_CLUSTER
295  таблетки    0.530486  ...    2                  0.0
421  препарат    0.537895  ...    2                  2.0
429  препарат    0.537895  ...    2                  2.0
454  препарат    0.537895  ...    2                  2.0

[4 rows x 23 columns]
          TOKEN  TOKEN_VECT  ...  NER  COREFERENCE_CLUSTER
21   лекарством    0.427442  ...    2                  0.0
475         его    0.475986  ...    3                  2.0

[2 rows x 23 columns]
          TOKEN  TOKEN_VECT  ...  NER  COREFERENCE_CLUSTER
446  последнему    0.523204  ...    2                442.0

[1 rows x 23 columns]
    TOKEN  TOKEN_VECT  IS_ANSWER  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
566    он    0.515951          0  ...          0.649308    3                536.0

[1 rows x 23 columns]
         TOKEN  TOKEN_VECT  ...  NER  COREFERENCE_CLUSTER
575   препарат    0.537895  ...    2                  0.0
587         он    0.515951  ...    3            

In [ ]:
df.loc[df['SENT_NUM'] != 0.0]

In [65]:
for i in range(len(df)):
  current_raw = df.iloc[[i]].loc[df['NER'] != 0.0]
  print(current_raw)
  if i == 3:
    break

Empty DataFrame
Columns: [TOKEN, TOKEN_VECT, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, HEAD_VECT, SENT_VECT, HEAD_CHILDS_VECT, NER, COREFERENCE_CLUSTER]
Index: []
Empty DataFrame
Columns: [TOKEN, TOKEN_VECT, IS_ANSWER, TOPIC_NUM, POST_NUM, SENT_NUM, TOKEN_NUM, ANIMACY, CASE, GENDER, PERSON, POS, DEPENDENCY, HEAD, HEAD_ANIMACY, HEAD_CASE, HEAD_GENDER, HEAD_POS, HEAD_VECT, SENT_VECT, HEAD_CHILDS_VECT, NER, COREFERENCE_CLUSTER]
Index: []
        TOKEN  TOKEN_VECT  ...  NER  COREFERENCE_CLUSTER
2  рисперидон    0.779845  ...  1.0                  2.0

[1 rows x 23 columns]
       TOKEN  TOKEN_VECT  IS_ANSWER  ...  HEAD_CHILDS_VECT  NER  COREFERENCE_CLUSTER
3  рисполепт    0.797031          0  ...          0.830899  1.0                  3.0

[1 rows x 23 columns]


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [1404]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [1405]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [1406]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'привился', 'прививки', 'решать', 'неизвестно', 'быстро', 'проводились',
        'испытания', 'аукнется', 'вакцина', 'организме'],
       dtype='object', length=2519),
 'HEAD_ANIMACY': Index(['inan', 'ani

In [1407]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [1408]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,0.779845,0,1,0,0,2,1,0,0,-1,1,2,1,0,0,0,1,0.878739,0.835993,0.808772,1.0,2.0
3,рисполепт,0.797031,0,1,0,0,3,0,1,1,-1,1,3,2,1,0,0,1,0.554854,0.835993,0.830899,1.0,3.0
4,сперидан,0.760111,0,1,0,0,4,-1,-1,0,-1,2,4,3,0,1,1,1,0.602089,0.835993,0.805134,1.0,4.0
21,лекарством,0.456023,0,1,1,1,10,0,5,2,-1,1,4,7,-1,-1,-1,3,0.520258,0.518605,0.660347,2.0,0.0
142,рисполепта,0.731182,0,1,1,13,7,0,0,1,-1,1,21,69,0,2,1,1,0.471137,0.639488,0.762393,1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12726,приходили,0.534609,0,15,0,10,1,-1,-1,-1,-1,0,0,2307,-1,-1,-1,0,0.510058,0.492714,0.630305,1.0,204.0
12785,завершению,0.453896,0,15,0,16,1,0,2,2,-1,1,10,1851,-1,-1,-1,0,0.390514,0.437342,0.503593,1.0,263.0
12854,к,0.415880,0,15,0,22,3,-1,-1,-1,-1,7,9,2332,1,2,1,1,0.626481,0.641312,0.428054,1.0,332.0
12876,стадию,0.542401,0,15,0,25,7,0,3,1,-1,1,1,1777,-1,-1,-1,0,0.475252,0.694040,0.622309,2.0,0.0


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [1409]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [1410]:
train = train.reset_index()

In [1411]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [1412]:
test = test.reset_index()

In [1413]:
test.head(5)

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,3673,препараты,0.468716,1,3,22,1,1789,0,0,0,-1,1,1,739,-1,-1,-1,5,0.658265,0.673832,0.677477,2.0,1795.0
1,2256,препаратов,0.464472,1,3,5,12,372,0,1,0,-1,1,21,569,0,5,1,1,0.420347,0.502091,0.589498,2.0,388.0
2,9144,химии,0.556986,1,7,1,18,241,0,1,1,-1,1,10,1699,-1,0,2,8,0.465183,0.564593,0.646650,2.0,0.0
3,7336,нейролептиками,0.589178,1,5,35,3,1134,0,5,0,-1,1,21,1459,0,2,1,1,0.418195,0.493534,0.665142,2.0,0.0
4,6526,препарат,0.525897,1,5,16,0,324,0,0,0,-1,1,1,1280,-1,-1,-1,0,0.583459,0.713328,0.764346,2.0,319.0


In [1414]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [1415]:
# test.loc[test['NER'] != 0]

In [1416]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [1417]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [1418]:
X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [1419]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [1420]:
X

,TOKEN_VECT,TOKEN_NUM,GENDER,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,0.519055,221,0,5,0.743045,3.0,218.0
1,0.584090,84,-1,-1,0.615114,2.0,0.0
2,0.454605,35,-1,16,0.525582,2.0,0.0
3,0.457997,62,-1,11,0.600316,2.0,6.0
4,0.471790,1660,1,5,0.739101,3.0,1657.0
...,...,...,...,...,...,...,...
226,0.476860,1568,1,5,0.616808,3.0,1582.0
227,0.525897,320,0,1,0.641136,2.0,663.0
228,0.468716,732,0,1,0.435643,2.0,0.0
229,0.525897,2721,0,1,0.703564,2.0,2711.0


In [1421]:
X_test.head(10)

,TOKEN_VECT,TOKEN_NUM,GENDER,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,0.468716,1789,0,1,0.673832,2.0,1795.0
1,0.464472,372,0,1,0.502091,2.0,388.0
2,0.556986,241,1,1,0.564593,2.0,0.0
3,0.589178,1134,0,1,0.493534,2.0,0.0
4,0.525897,324,0,1,0.713328,2.0,319.0
5,0.406463,72,2,1,0.584244,2.0,0.0
6,0.459385,1032,0,1,0.497320,2.0,1081.0
7,0.505452,512,0,5,0.484634,3.0,507.0
8,0.525897,265,0,1,0.625485,2.0,287.0
9,0.525897,3113,0,1,0.519936,2.0,3125.0


In [1422]:
y

0       218.0
1         0.0
2         0.0
3         6.0
4      1657.0
        ...  
226    1582.0
227     663.0
228       0.0
229    2711.0
230       0.0
Name: COREFERENCE_CLUSTER, Length: 231, dtype: float64

Обучаем модели.

In [1423]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.603448275862069

In [1424]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6379310344827587

In [1425]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.5344827586206896

In [1426]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)

1.0

In [1427]:
from sklearn import svm

In [1428]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.39655172413793105

In [1429]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=7, weights='distance',algorithm='ball_tree',leaf_size=40, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6724137931034483

In [1430]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [1431]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [1432]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [1433]:
# my_eval(list(naive_pred), list(y_test))

In [1434]:
my_eval(list(pred_knn), list(y_test))

17  true positive
17  false positive
22  true negative
2  false negative
---------
0.5  presicion
0.8947368421052632  recall
0.6415094339622641  f-score


In [1435]:
my_eval(list(pred_entropy), list(y_test))

14  true positive
20  false positive
23  true negative
1  false negative
---------
0.4117647058823529  presicion
0.9333333333333333  recall
0.5714285714285713  f-score


In [1436]:
my_eval(list(pred), list(y_test))

12  true positive
22  false positive
23  true negative
1  false negative
---------
0.35294117647058826  presicion
0.9230769230769231  recall
0.5106382978723405  f-score


In [1437]:
my_eval(list(pred_randf), list(y_test))

8  true positive
26  false positive
23  true negative
1  false negative
---------
0.23529411764705882  presicion
0.8888888888888888  recall
0.3720930232558139  f-score


In [1438]:
my_eval(list(pred_regr), list(y_test))

2  true positive
56  false positive
0  true negative
0  false negative
---------
0.034482758620689655  presicion
1.0  recall
0.06666666666666667  f-score


In [1439]:
my_eval(list(pred_svm), list(y_test))

0  true positive
12  false positive
23  true negative
23  false negative
---------
0.0  presicion
0.0  recall
0  f-score
